This is the first, simple demonstration of the development version of this code.

In [ ]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

import libst

Show propagating wave.

In [ ]:
grid = libst.Grid(0, 4*2*np.pi, 4*64)
cfl = 1
dx = (grid.xmax - grid.xmin) / grid.ncelm
dt = dx * cfl
svr = libst.LinearScalarSolver(grid=grid, time_increment=dt)

# Initialize
for e in svr.selms(odd_plane=False):
    if e.xctr < 2*np.pi or e.xctr > 2*2*np.pi:
        v = 0
        dv = 0
    else:
        v = np.sin(e.xctr)
        dv = np.cos(e.xctr)
    e.set_so0(0, v)
    e.set_so1(0, dv)
    
plt.figure(figsize=(15,10))
plt.plot(svr.xctr() / np.pi, svr.get_so0(0), '-')

svr.setup_march()
svr.march_alpha2(50)
plt.plot(svr.xctr() / np.pi, svr.get_so0(0), '-')

In [ ]:
grid = libst.Grid(0, 4*2*np.pi, 4*64)
cfl = 1
dx = (grid.xmax - grid.xmin) / grid.ncelm
dt = dx * cfl
svr = libst.LinearScalarSolver(grid=grid, time_increment=dt)

# Initialize
for e in svr.selms(odd_plane=False):
    if e.xctr < 2*np.pi or e.xctr > 2*2*np.pi:
        v = 0
        dv = 0
    else:
        v = np.sin(e.xctr)
        dv = np.cos(e.xctr)
    e.set_so0(0, v)
    e.set_so1(0, dv)

def animate(nsteps, ninterval):
    fig, ax = plt.subplots(figsize=(15,10))

    ax.set_xlim((svr.selm(0).x/np.pi, svr.selm(svr.grid.ncelm).x/np.pi))
    ax.set_ylim((-1.2, 1.2))
    ax.grid()

    line, = ax.plot([], [], "-")

    def init():
        svr.setup_march()
        line.set_data(svr.xctr()/np.pi, svr.get_so0(0))
        return (line,)

    def animate(i):
        if i != 0:
            svr.march_alpha2(ninterval)
        line.set_data(svr.xctr()/np.pi, svr.get_so0(0))
        return (line,)

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=nsteps+1, interval=100,
                                   blit=True)
    return HTML(anim.to_jshtml())
            
#animate(100, 10) # Don't check in the animation to git, since it's big.